# Bike Sharing Demand (Hyperparameter Tunning)
https://www.kaggle.com/c/bike-sharing-demand

In [ ]:
import pandas as pd

## Load Dataset

In [ ]:
# parse_dates : 'datetime' 컬럼을 python date type 으로 처리하기 위해
train = pd.read_csv("data/train.csv", parse_dates=["datetime"])

print(train.shape)
train.head()

## Explore

In [ ]:
# 차트를 jupyter notebook에 출력해서 보기 위한 명령어
%matplotlib inline

import seaborn as sns
import matplotlib.pyplot as plt

### Explore datetime

In [ ]:
# datatime 을 python date type 으로 load 했기 때문에 '.dt' 속성을 이용해서 년/월/일/시/분/초/요일 등의 정보에 접근 가능함
# datatime 으로 부터 년/월/일/시/분/초 컬럼 추가
train["datetime-year"] = train["datetime"].dt.year
train["datetime-month"] = train["datetime"].dt.month
train["datetime-day"] = train["datetime"].dt.day
train["datetime-hour"] = train["datetime"].dt.hour
train["datetime-minute"] = train["datetime"].dt.minute
train["datetime-second"] = train["datetime"].dt.second

print(train.shape)
train[["datetime", "datetime-year", "datetime-month", "datetime-day", "datetime-hour", "datetime-minute", "datetime-second"]].head()

In [ ]:
# 2 X 3 subplot 선언
figure, ((ax1, ax2, ax3), (ax4, ax5, ax6)) = plt.subplots(nrows=2, ncols=3)
# subplot size 선언
figure.set_size_inches(18, 8)
# time 관련 feature 들과 대여 수(count) 관계를 barplot 을 이용해 확인
sns.barplot(data=train, x="datetime-year", y="count", ax=ax1)
sns.barplot(data=train, x="datetime-month", y="count", ax=ax2)
sns.barplot(data=train, x="datetime-day", y="count", ax=ax3)
sns.barplot(data=train, x="datetime-hour", y="count", ax=ax4)
sns.barplot(data=train, x="datetime-minute", y="count", ax=ax5)
sns.barplot(data=train, x="datetime-second", y="count", ax=ax6)

** Lesson Learned **
  * **datetime-minute**와 **datetime-second**는 현재 기록되고 있지 않다. 그러므로 사용할 필요가 없다.
  * train.csv와 test.csv는 **datetime-day**를 기준으로 나뉘어져 있다. 그러므로 **datetime-day**를 feature로 사용해서는 안 된다.

### Explore hour - workingday

In [ ]:
# 2 X 1 subplot 선언
figure, (ax1, ax2) = plt.subplots(nrows=2, ncols=1)
figure.set_size_inches(18, 8)

# 시간과 대여 수(count)의 관계를 pointplot 을 이용해 확인
sns.pointplot(data=train, x="datetime-hour", y="count", ax=ax1)
# hue="workingday" 를 추가해서 주중 시간대와, 주말 시간대별 대여수 관계를 확인
sns.pointplot(data=train, x="datetime-hour", y="count", hue="workingday", ax=ax2)

** Lesson Learned **
  * 주중(workingday==1)에는 출근 시간과 퇴근 시간에 자전거를 많이 대여한다.
  * 주말(workingday==0)에는 오후 시간에 자전거를 많이 대여한다.
  * 주중(월,화,수,목,금)이 주말(토,일)보다 많기 때문에, 두 개를 나눠서 보지 않으면 주말의 특성을 파악할 수 없다.

### Explore hour - dayofweek

In [ ]:
# 요일(dayofweek) 컬럼 추가 (0 = monday, ~ 6 = sunday) 
train["datetime-dayofweek"] = train["datetime"].dt.dayofweek

print(train.shape)
train[["datetime", "datetime-dayofweek"]].head()

In [ ]:
# 2 X 1 subplot 선언
figure, (ax1, ax2) = plt.subplots(nrows=2, ncols=1)
figure.set_size_inches(18, 8)

# 주중/주말 시간대별 대여수 차이
sns.pointplot(data=train, x="datetime-hour", y="count", hue="workingday", ax=ax1)
# 요일 시간대별 대여수 차이
sns.pointplot(data=train, x="datetime-hour", y="count", hue="datetime-dayofweek", ax=ax2)

** Lesson Learned **
  * 금요일(workingday==4)는 주중이지만, 아주 약간 주말의 특성을 반영하고 있다.
  * 비슷하게 월요일(workingday==0)도 아주 약간 주말의 특성을 반영하고 있다.
  * 사람들이 휴가를 월요일과 금요일에 사용하기 때문이라고 추측할 수 있다.

### Concatenate year and month

In [ ]:
# datetime 을 받아서 '년-월' 값을 만들어 주는 함수 정의
def concatenate_year_month(datetime):
    return "{0}-{1}".format(datetime.year, datetime.month)
# 년-월 컬럼 추가 (년-월 별 대여량 차이 확인 하기 위해서)
train["datetime-year_month"] = train["datetime"].apply(concatenate_year_month)

print(train.shape)
train[["datetime", "datetime-year_month"]].head()

In [ ]:
# 1 X 1 subplot 선언
figure, (ax1, ax2) = plt.subplots(nrows=1, ncols=2)
figure.set_size_inches(18, 4)
# 연도별 대여수 차이 확인
sns.barplot(data=train, x="datetime-year", y="count", ax=ax1)
# 월별 대여수 차이 확인
sns.barplot(data=train, x="datetime-month", y="count", ax=ax2)

# 하나짜리 subplot 선언 (차트 하나를 원하는 사이즈로 출력하고 싶을 때 사용하는 Tip)
figure, ax3 = plt.subplots(nrows=1, ncols=1)
figure.set_size_inches(18, 4)
# 위에서 새로 추가한 년-월 별 대여수 차이 확인
sns.barplot(data=train, x="datetime-year_month", y="count", ax=ax3)

** Lesson Learned **
  * 2011년 12월과 2012년 1월의 자전거 대여량을 비슷하지만, 두 개를 따로 놓고 보면 이를 알 수 없다.
  * 2011년에는 8월부터 대여량이 감소하고, 2012년에는 7월부터 대여량이 감소한다. 마찬가지로 따로 놓고 보면 이를 알 수 없다.

## Reload Dataset

In [ ]:
# 전처리 하기위해 train 원본 데이터를 다시 load
train = pd.read_csv("data/train.csv", parse_dates=["datetime"])

print(train.shape)
train.head()

In [ ]:
# test 데이터도 전처리를 같이 해야 하기 때문에 함께 load
test = pd.read_csv("data/test.csv", parse_dates=["datetime"])

print(test.shape)
test.head()

## Preprocessing

### Parse datetime

In [ ]:
# train 데이터에 년/월/일/시간/요일 컬럼 추가
train["datetime-year"] = train["datetime"].dt.year
train["datetime-month"] = train["datetime"].dt.month
train["datetime-day"] = train["datetime"].dt.day
train["datetime-hour"] = train["datetime"].dt.hour
train["datetime-dayofweek"] = train["datetime"].dt.dayofweek

print(train.shape)
train[["datetime", "datetime-year", "datetime-month", "datetime-day", "datetime-hour", "datetime-dayofweek"]].head()

In [ ]:
# test 데이터에 년/월/일/시간/요일 컬럼 추가
test["datetime-year"] = test["datetime"].dt.year
test["datetime-month"] = test["datetime"].dt.month
test["datetime-day"] = test["datetime"].dt.day
test["datetime-hour"] = test["datetime"].dt.hour
test["datetime-dayofweek"] = test["datetime"].dt.dayofweek

print(test.shape)
test[["datetime", "datetime-year", "datetime-month", "datetime-day", "datetime-hour", "datetime-dayofweek"]].head()

## Train

In [ ]:
# train 시킬 feature 선언
feature_names = ["season", "holiday", "workingday", "weather",
                 "temp", "atemp", "humidity", "windspeed",
                 "datetime-year", "datetime-hour", "datetime-dayofweek"]

feature_names

In [ ]:
# train 시킬 데이터셋(feature) 준비 : X_train
X_train = train[feature_names]

print(X_train.shape)
X_train.head()

In [ ]:
# 예측(predict) 할 test 데이터셋 준비 : X_test
X_test = test[feature_names]

print(X_test.shape)
X_test.head()

In [ ]:
# train 시킬 때 사용할 label(target, 종속변수) 컬럼 선택
label_name = "count"

# train 시킬 때 사용할 label(target, 종속변수) 데이터셋 준비
y_train = train[label_name]

print(y_train.shape)
y_train.head()

In [ ]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(random_state=37)
model

## Hyperparameter Tuning

https://www.analyticsvidhya.com/blog/2015/06/tuning-random-forest-model/

### Case 1 - Grid Search

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.cross_validation import cross_val_score

n_estimators = 300

max_depth_list = [5, 10, 15, 20, 25, 30, 35, 40, 45, 50]
max_features_list = [0.1, 0.3, 0.5, 0.7, 0.9, 1.0]

hyperparameters_list = []

for max_depth in max_depth_list:
    for max_features in max_features_list:
        model = RandomForestRegressor(n_estimators=n_estimators,
                                      max_depth=max_depth,
                                      max_features=max_features,
                                      random_state=37,
                                      n_jobs=-1)

        score = cross_val_score(model, X_train, y_train, cv=20, \
                                scoring='neg_mean_absolute_error').mean()

        hyperparameters_list.append({
            'score': score,
            'n_estimators': n_estimators,
            'max_depth': max_depth,
            'max_features': max_features,
        })

        print("Score = {0:.5f}".format(score))

hyperparameters_list

In [ ]:
hyperparameters_list = pd.DataFrame.from_dict(hyperparameters_list)
hyperparameters_list = hyperparameters_list.sort_values(by="score")

print(hyperparameters_list.shape)
hyperparameters_list.head()

### Case 2 - Random Search (Coarse Search)

In [ ]:
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.cross_validation import cross_val_score

hyperparameters_list = []

n_estimators = 300
num_epoch = 100

for epoch in range(num_epoch):
    max_depth = np.random.randint(low=2, high=100)
    max_features = np.random.uniform(low=0.1, high=1.0)

    model = RandomForestRegressor(n_estimators=n_estimators,
                                  max_depth=max_depth,
                                  max_features=max_features,
                                  random_state=37,
                                  n_jobs=-1)

    score = cross_val_score(model, X_train, y_train, cv=20, \
                            scoring='neg_mean_absolute_error').mean()

    hyperparameters_list.append({
        'score': score,
        'n_estimators': n_estimators,
        'max_depth': max_depth,
        'max_features': max_features,
    })

    print("Score = {0:.5f}".format(score))

hyperparameters_list = pd.DataFrame.from_dict(hyperparameters_list)
hyperparameters_list = hyperparameters_list.sort_values(by="score")

print(hyperparameters_list.shape)
hyperparameters_list.head()

### Case 2 - Random Search(Finer Search)

In [ ]:
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.cross_validation import cross_val_score

hyperparameters_list = []

n_estimators = 300
num_epoch = 100

for epoch in range(num_epoch):
    max_depth = np.random.randint(low=10, high=70)
    max_features = np.random.uniform(low=0.4, high=1.0)

    model = RandomForestRegressor(n_estimators=n_estimators,
                                  max_depth=max_depth,
                                  max_features=max_features,
                                  random_state=37,
                                  n_jobs=-1)

    score = cross_val_score(model, X_train, y_train, cv=20, \
                            scoring='neg_mean_absolute_error').mean()

    hyperparameters_list.append({
        'score': score,
        'n_estimators': n_estimators,
        'max_depth': max_depth,
        'max_features': max_features,
    })

    print("Score = {0:.5f}".format(score))

hyperparameters_list = pd.DataFrame.from_dict(hyperparameters_list)
hyperparameters_list = hyperparameters_list.sort_values(by="score")

print(hyperparameters_list.shape)
hyperparameters_list.head()

In [ ]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=3000,
                              max_depth=83,
                              max_features=0.851358,
                              random_state=37,
                              n_jobs=-1)
model

## Score

#### a : 정답, P : 측정값

$|P-a|$ = mean absolute error (MAE)


$|P-a|^2$ = mean squared error (MSE)


$ \sqrt{(p-a)^2}$ = root mean squared error (RMSE)


$ \sqrt{\frac{1}{n} \sum_{i=1}^n (\log(p_i + 1) - \log(a_i+1))^2 } $ = root mean squared log error (RMSLE)


- 차이가 크면 클수록 페널티를 주고 싶으면 MSE 를 사용 함

- 차이가 크면 클루록 페널티가 줄어들게 하고 싶으면 RMSLE 사용 함 (RMSLE 로직은 직접 구현해야 함)

In [ ]:
from sklearn.cross_validation import cross_val_score

score = cross_val_score(model, X_train, y_train, cv=20, \
                         scoring='neg_mean_absolute_error').mean()

print("Score = {0:.5f}".format(-score))

## Train

In [ ]:
model.fit(X_train, y_train)

In [ ]:
predictions = model.predict(X_test)

print(predictions.shape)
predictions

#### Calculate feature importances

In [ ]:
importances = model.feature_importances_
importance_by_feature = list(zip(feature_names, importances))
importance_by_feature

In [ ]:
# subplots 를 이용해서 차트 하나의 가로/세로 사이즈를 조정하고 싶을 경우 응용
figure, ax1 = plt.subplots(nrows=1, ncols=1)
figure.set_size_inches(18, 4)

importance_df = pd.DataFrame(importance_by_feature, columns=['feature', 'importance'])
sns.barplot(data=importance_df, x='feature', y='importance', ax=ax1)

## Submit

In [ ]:
submission = pd.read_csv("data/sampleSubmission.csv")

submission["count"] = predictions

print(submission.shape)
submission.head()

In [ ]:
# 저장할 파일을 구분하기 위해 파일명에 timestamp 정보 추가 하기 위한 작업 
from datetime import datetime

current_date = datetime.now()
current_date = current_date.strftime("%Y-%m-%d_%H-%M-%S")

description = "bike-random-forest"

filename = "{date}_{desc}_{score}.csv".format(date=current_date, desc=description, score="{0:.5f}".format(score))
filepath = "data/{filename}".format(filename=filename)

submission.to_csv(filepath, index=False)